In [ ]:
#| default_exp main

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export

from functools import partial
from pathlib import Path
import shutil

from bedmap.prepare_images import df_images_from_pattern
from bedmap.validate_images import validate_images
from bedmap.create_thumbnails import create_thumbnails_col
from bedmap.embed_images import create_embeddings_col
from bedmap.create_umap_layout import create_umap_col
from bedmap.scaffold_output import copy_web_assets
from bedmap.manifests import write_manifest
from bedmap.config import Cfg

/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/pydantic_settings/main.py:425: UserWarning: Config key `pyproject_toml_table_header` is set in model_config but will be ignored because no PyprojectTomlConfigSettingsSource source is configured. To use this config key, add a PyprojectTomlConfigSettingsSource source to the settings sources via the settings_customise_sources hook.
  self._settings_warn_unused_config_keys(sources, self.model_config)
/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/umap/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


ImportError: cannot import name 'write_manifest' from 'bedmap.manifests' (/home/willsa/git/bedmap-dev/src/bedmap/manifests.py)

## 2. All-steps pipeline

In [ ]:
#| export

def process_images_to_umap(cfg):
    steps = [validate_images,
        partial(create_thumbnails_col, height=cfg.thumbnail_size),
    ]
    if cfg.model_name is not None:
        steps.append(partial(create_embeddings_col, model_name=cfg.model_name,
        batch_size=2))
    if isinstance(cfg.umap_spec, list):
        for spec in cfg.umap_spec:
            steps.append(partial(create_umap_col, **cfg.umap_spec.model_dump()))
    elif cfg.umap_spec:
        steps.append(partial(create_umap_col, **cfg.umap_spec.model_dump()))

    return steps

In [ ]:

#| export

def run_recipe(cfg):
    df = df_images_from_pattern(cfg.paths.image_glob)
    for fn in process_images_to_umap(cfg):
        df = fn(df)
    copy_web_assets(cfg.paths.output_dir)
    write_manifest(cfg)
    return df

In [ ]:
#| hide

cfg = Cfg()

# TODO: figure out how to get this config from pyproject.toml
test_root = Path().resolve().parent / "tests"

cfg.paths.output_dir = test_root / "test-outputs/output_00"
cfg.paths.image_glob = test_root / "test-data/smithsonian_butterflies_10/jpgs/*.jpg"

run_recipe(cfg)

shutil.rmtree(cfg.paths.output_dir)

Checking size_nontrivial


🗡️ 🐟 InMemorySource: 00:00 

🗡️ 🐟 Project: 00:00 

🗡️ 🐟 Filter: 00:00 

🗡️ 🐟 Count: 00:00 

...

🗡️ 🐟 Filter: 00:00 

🗡️ 🐟 Hash Join Build: 00:00 

🗡️ 🐟 Inner Hash Join Probe: 00:00 

🗡️ 🐟 Project: 00:00 

/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/umap/umap_.py:2344: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


In [ ]:
#| hide

import nbdev
nbdev.nbdev_export()